# Project 2

Consider the truss assembly pictured in the figure below.

![img](images/truss.png)

Your job as an engineer is to choose the structural steel required to ensure the truss assembly shown in the figure will not fail under a variety of loading conditions.  The external forces applied to the truss are always applied at the locations $B$ and $D$ in the directions shown.  The external forces can be summarized as

\begin{equation}
\vec{F}=\left[
\begin{array}{c}
{B}_y \\
{D}_x \\
{D}_y
\end{array}
\right]
\end{equation}

In order to determine which structural steel to use we must first determine the force in each truss member under a variety of loading conditions.  A survey of the applications for this truss has resulted in the following possible loading conditions

\begin{equation}
\vec{F}_1=\left[
\begin{array}{c}
400 \mathrm{N} \\
600 \mathrm{N} \\
0 \mathrm{N}
\end{array}
\right],
%
\vec{F}_2=\left[
\begin{array}{c}
700 \mathrm{N} \\
100 \mathrm{N} \\
200 \mathrm{N}
\end{array}
\right],
%
\vec{F}_3=\left[
\begin{array}{c}
100 \mathrm{N} \\
1000 \mathrm{N} \\
100 \mathrm{N}
\end{array}
\right],
%
\vec{F}_4=\left[
\begin{array}{c}
0 \mathrm{N} \\
500 \mathrm{N} \\
500 \mathrm{N}
\end{array}
\right],
%
\vec{F}_5=\left[
\begin{array}{c}
500 \mathrm{N} \\
0 \mathrm{N} \\
1500 \mathrm{N}
\end{array}
\right],
\end{equation}
\begin{equation}
%
\vec{F}_6=\left[
\begin{array}{c}
1200 \mathrm{N} \\
0 \mathrm{N} \\
100 \mathrm{N}
\end{array}
\right],
%
\vec{F}_7=\left[
\begin{array}{c}
1600 \mathrm{N} \\
0 \mathrm{N} \\
0 \mathrm{N}
\end{array}
\right],
%
\vec{F}_8=\left[
\begin{array}{c}
0 \mathrm{N} \\
0 \mathrm{N} \\
1600 \mathrm{N}
\end{array}
\right],
%
\vec{F}_9=\left[
\begin{array}{c}
0 \mathrm{N} \\
1600 \mathrm{N} \\
0 \mathrm{N}
\end{array}
\right],
%
\vec{F}_{10}=\left[
\begin{array}{c}
500 \mathrm{N} \\
500 \mathrm{N} \\
500 \mbox{N}
\end{array}
\right].
\end{equation}

Using either Gaussian elimination or LU Decomposition (you can use your solution from [assignment13](https://github.com/PGE310-Students/assignment13) and/or [assignment14](https://github.com/PGE310-Students/assignment14), or the built in NumPy/SciPy implementations) complete the function `compute_largest_member_force` below to efficiently solve for the forces in all members of the truss.  Using this information determine what the maximum load (compressive or tensile) is in any member for each loading configuration, the largest force (in magnitude) in any member, and which member contains the largest force.  

The function `compute_largest_member_force` should take a two-dimensional NumPy array as the input argument.  Each row of the input array corresponds to one of the loading conditions $F_1, F_2, \dots, F_{10}$.  So the array should be a 3 by 10 for these particular loads.  

The function should return a three-tuple that contains `(member_id_string, force_id_string, magnitude_largest_force)` where `member_id_string` is a string that labels the member that contains the largest force, so one of `'AD', 'AB', 'BD', 'CD',` or `'BC'`.  `force_id_string` is a string that labels the loading condition that gives rise to the maximum force so one of `'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9',` or `'F10'`. `magnitude_largest_force` is the magnitude of the largest force in any member.  So an example of what the function should return is `('AB', 'F1', 102.5)` in that order!

If you use your assignment13 or 14 codes and import them after converting to Python, make sure you add those files to the repository when you commit them.

**Hint:** You must first solve for the reaction at the pins of the entire truss in equilibrium.  These reaction forces along with the externally applied loads can then be used with the Method of Sections or the Method of Joints to write the equilibrium equations for each member of the truss.

In [15]:
import numpy as np
from assignment14 import Matrix, LU
from assignment13 import LinearSystem

def compute_largest_member_force(forces):
    
    #get number of inputs, columns 
    n = forces.shape[0]
    
    #init trig
    theta = np.arctan([4/3])
    sin = np.sin(theta).item()
    cos = np.cos(theta).item()

    #create zero 8 by n matrix for all solutions
    #to be replaced with solutions as rows
    all_sols = np.zeros((n, 8))
    
    #loop over all columns of 'forces', calculate solutions 
    #to solve for 8 unknowns, append them as rows to all_sols
    for i in range(n):

        #pull first row (column) of inputs from test forces and make column
        input_vector = forces[i, :].transpose()
        #input_vector = forces[:, i]
        
        #from input_vector define input forces
        By = input_vector[0].item()
        Dx = input_vector[1].item()
        Dy = input_vector[2].item()
        
        #create right side vector
        #right_side = np.array([0, 0, 0, By, 0, 0, -1*Dx, -1*Dy])
        right_side = np.array([[0], [0], [0], [By], [0], [0], [-1*Dx], [Dy]])
        
        #create the input mat for decomposition
        input_mat = np.array([[0, 0, 0, 1, cos, 0, 0, 0], [0, 0, 1, 0, sin, 0, 0, 0], [0, 0, 0, 0, -1*cos, cos, 1, 0], [0, 0, 0, 0, -1*sin, -1*sin, 0, 0], [1, 0, 0, 0, 0, 0, -1, 0], [0, 1, 0, 0, 0, 0, 0, -1], [0, 0, 0, -1, 0, -1*cos, 0, 0], [0, 0, 0, 0, 0, sin, 0, 1]])
    
        #create linear system
        LS = LinearSystem(input_mat, right_side)
        
        #solve and replace row number
        row_sol = LS.gauss_solve()
        all_sols[i] = row_sol
    
    #pull only tensions
    tension_sols_NaA = all_sols[:, 3:]
    #tension_sols = all_sols[:, 3:]
    tension_sols = np.absolute(tension_sols_NaA)
    
    #find max value
    #max_sol = np.amax(np.absolute(tension_sols))
    max_sol = np.amax(tension_sols)
    
    #find the location of max
    max_location = np.where(tension_sols == max_sol)
    row = max_location[0].item()
    col = max_location[1].item()
    
    #array of all members
    all_members = np.array(["AD", "AB", "BD", "BC", "CD"])
    
    #returned tuple
    return_var = (all_members[col].item(), "F{}".format(row+1), max_sol)
    
    return return_var #max_location #input_vector # forces #(member_id_string, force_id_string, magnitude_largest_force)

In [16]:
#TEST FORCES ARE TRANSPOSED
#test_forces = np.array([[400, 700, 100, 0, 500, 1200, 1600, 0, 0, 500], [600, 100, 1000, 500, 0, 0, 0, 0, 1600, 500], [0, 200, 100, 500, 1500, 100, 0, 1600, 0, 500]]).transpose()
#test = compute_largest_member_force(test_forces)
#print(test)